In [1]:
import boto3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import joblib

In [2]:
# Load data from S3 (ensure you have the correct S3 bucket and file path)
s3 = boto3.client('s3')
s3.download_file('noe-processed-data-by-challenge', 'fact/transactions/part-00000-08c1836c-e295-4729-95a9-67b56c05d840-c000.snappy.parquet', 'part-00000-08c1836c-e295-4729-95a9-67b56c05d840-c000.snappy.parquet')

In [4]:
#Load parquet using pyarrow engine
df = pd.read_parquet('part-00000-08c1836c-e295-4729-95a9-67b56c05d840-c000.snappy.parquet', engine='pyarrow')

In [5]:
df

,Fecha,Saldo_Inversion,saldo_clientes,num_ingresos_hoy,num_egresos_hoy,Saldo_Flujos
0,2022-12-01,3.228325e+09,4.214973e+09,12931,43273,1.064952e+09
1,2022-12-02,3.629307e+09,3.881660e+09,12556,84912,1.035548e+08
2,2022-12-03,3.581642e+09,3.882157e+09,530,2021,1.037474e+08
3,2022-12-05,3.181642e+09,3.507273e+09,10131,29326,1.671188e+08
4,2022-12-06,3.182190e+09,3.420021e+09,10019,22640,7.928007e+07
...,...,...,...,...,...,...
372,2024-01-10,5.567876e+09,5.655550e+09,9657,31526,2.617011e+08
373,2024-01-11,5.419947e+09,5.579618e+09,12237,39282,3.407682e+08
374,2024-01-12,5.269947e+09,5.546198e+09,18250,130252,4.573260e+08
375,2024-01-13,5.273781e+09,5.540931e+09,875,7100,4.353983e+08


In [7]:
# Prepare data
X = np.array(range(len(df['Saldo_Inversion'])))
y = df['Saldo_Inversion'].to_numpy()

In [ ]:
X.shape, y.shape

In [9]:
# Train model
final_model = LinearRegression()
final_model.fit(X.reshape(-1, 1), y)

LinearRegression()

In [15]:
# Save model to S3
joblib.dump(final_model, 'model_saldo_inversion.joblib')
s3.upload_file('model_saldo_inversion.joblib', 'noe-processed-data-by-challenge', 'models/saldos/model_saldo_inversion.joblib')